# 빈도수 기반 텍스트 표현 (BoW, TF-IDF)

## BOW 기반의 카운트 벡터 생성

* 사이킷런의 CountVectorizer 클래스 활용 연습
    * sklearn 설치 : **pip install scikit-learn**

### 영어 문서 카운트 벡터 추출

In [1]:
sample_corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?'
]

In [4]:
# Vectorizer 객체 생성
from sklearn.feature_extraction.text import CountVectorizer
cVectorizer = CountVectorizer()

# 특징 집합 생성
cVectorizer.fit(sample_corpus)
print(cVectorizer.get_feature_names_out())

# 카운트 벡터 생성
sample_dtm = cVectorizer.transform(sample_corpus)

# DTM 출력
print(sample_dtm.toarray())


['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [5]:
# 참고
feature_set = cVectorizer.vocabulary_
print(feature_set)

# index 0부터 출력
print(sorted(feature_set.items(), key=lambda x : x[1]))

{'this': 8, 'is': 3, 'the': 6, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 7, 'one': 4}
[('and', 0), ('document', 1), ('first', 2), ('is', 3), ('one', 4), ('second', 5), ('the', 6), ('third', 7), ('this', 8)]


### 한글 문서 카운트 벡터 추출

In [7]:
sample_corpus = [
    '자연어처리 강의를 시작하겠습니다.',
    '자연어처리는 재미있습니다.',
    '밥을 먹고 강의를 듣고 있습니다.',
    '이번 자연어처리 강의는 한국어 자연어처리입니다.' 
]

In [13]:
# 한국어 tokenizer 정의
from konlpy.tag import Okt
def my_tokenizer(text):
    t = Okt()
    return t.nouns(text)

# Vectorizer 객체 생성
my_stopwords = ['이번']

from sklearn.feature_extraction.text import CountVectorizer
#cVectorizer = CountVectorizer(tokenizer=t.nouns)
cVectorizer = CountVectorizer(tokenizer=my_tokenizer, stop_words=my_stopwords)

# 특징 집합 생성
cVectorizer.fit(sample_corpus)
print(cVectorizer.get_feature_names_out())
print(cVectorizer.vocabulary_)

# 카운트 벡터 생성
sample_dtm = cVectorizer.transform(sample_corpus)

# DTM 출력
print(sample_dtm.toarray())

['강의' '밥' '시작' '자연어' '처리' '한국어']
{'자연어': 3, '처리': 4, '강의': 0, '시작': 2, '밥': 1, '한국어': 5}
[[1 0 1 1 1 0]
 [0 0 0 1 1 0]
 [1 1 0 0 0 0]
 [1 0 0 2 2 1]]


In [ ]:
# Vectorizer에 사용할 한국어 tokenizer 정의 (정제, 불용어제거 등의 기능 추가)
def my_tokenizer(text):
    t = Okt()

    my_tags = ['Noun', 'Verb', 'Adjective']
    tokens = [word for word, tag in t.pos(text) if tag in my_tags]
    return tokens

In [ ]:
# 사용자 정의 tokenizer를 사용하여 Vectorizer 객체 생성

# 특징 집합 생성


# 카운트 벡터 생성


# DTM 출력


## TF-IDF 벡터 생성
* sklearn.feature_extraction.text.TfidfVectorizer 사용
1. 한글 토크나이저 정의
2. 특징 추출 모델 생성 : Vectorizer -> fit()
3. 문서별 특징 벡터 추출 : transform()

### 한글 토크나이저 정의

In [ ]:
# corpus의 크기가 큰 경우 Okt 사용 추천 (Komoran의 경우 10000문장이 넘어 가면 너무 오래걸림)
from konlpy.tag import Okt

def my_tokenizer(text):
    t = Okt()
    my_tags = ['Noun', 'Verb', 'Adjective']
    tokens = [word for word, tag in t.pos(text) if tag in my_tags]
    return tokens

In [14]:
# 정의한 tokenizer로 1~2문서 확인
my_tokenizer(sample_corpus[1])

['자연어', '처리']

### 특징 추출 모델 생성 : TfIdfVectorizer

In [16]:
# TfIdfVectorizer 객체 생성 (tokenizer 지정, 최대 단어 수지 지정)
from sklearn.feature_extraction.text import TfidfVectorizer
tVectorizer = TfidfVectorizer(tokenizer=my_tokenizer, max_features=1000)

In [17]:
# 특징 집합과 특징 벡터 계산을 위한 데이터 추출
tVectorizer.fit(sample_corpus)
print(tVectorizer.get_feature_names_out())

['강의' '밥' '시작' '이번' '자연어' '처리' '한국어']


C:\Users\User\anaconda3\envs\deepnlpenv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 문서별 특징 벡터 추출

In [18]:
# 특징 벡터 추출
sample_tfidf_dtm = tVectorizer.transform(sample_corpus)
print(sample_tfidf_dtm.toarray())

[[0.42817512 0.         0.67081906 0.         0.42817512 0.42817512
  0.        ]
 [0.         0.         0.         0.         0.70710678 0.70710678
  0.        ]
 [0.53802897 0.84292635 0.         0.         0.         0.
  0.        ]
 [0.26813356 0.         0.         0.42008304 0.53626713 0.53626713
  0.42008304]]


## 다음(Daum) 영화 리뷰 TF-IDF 벡터 추출

In [20]:
# csv 파일 → DataFrame으로 업로드
import pandas as pd
movie_df = pd.read_csv('./data/daum_movie_review.csv')
movie_df.head()
review_corpus = movie_df.review
review_corpus[:10]

0                               돈 들인건 티가 나지만 보는 내내 하품만
1         몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.
2    이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...
3                                  이 정도면 볼만하다고 할 수 있음!
4                                                 재미있다
5                                             나는 재밌게 봄
6                                        0.5점은 줄 수 없냐?
7                       헐..다 죽었어....나중에 앤트맨 보다가도 깜놀...
8                                                충격 결말
9                                                  응집력
Name: review, dtype: object

In [21]:
# Vectorizer 객체 생성
from sklearn.feature_extraction.text import TfidfVectorizer
review_tv = TfidfVectorizer(tokenizer=my_tokenizer, max_features=1000)


In [22]:
# Vectorizer 모델 생성
review_tv.fit(review_corpus)

C:\Users\User\anaconda3\envs\deepnlpenv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=1000,
                tokenizer=<function my_tokenizer at 0x0000013C7FAC8EE0>)

In [25]:
# 특징 집합 구성 단어 확인
vocab = review_tv.get_feature_names_out()
print(vocab[100:200])

['귀인' '그' '그것' '그게' '그날' '그냥' '그닥' '그대로' '그동안' '그때' '그래픽' '극장' '극적' '근래'
 '글' '글쎄' '급' '기' '기담' '기대' '기도' '기법' '기본' '기분' '기사' '기술' '기억' '기자' '기존'
 '기준' '긴장' '긴장감' '길' '김' '김동욱' '김용화' '김자' '김향기' '깊이' '깜짝' '깡패' '깨알' '꼭'
 '꽤' '꿀잼' '꿈' '끝' '끼리' '나' '나라' '나름' '나머지' '나이' '나중' '난' '난리' '날' '남' '남녀'
 '남아' '남자' '남편' '남한' '낫다' '내' '내내' '내년' '내용' '너' '년' '노' '노래' '노력' '노스'
 '노잼' '놈' '누가' '누구' '누군가' '눈' '눈물' '눈물샘' '느낌' '는걸' '늘' '능력' '니' '다' '다가'
 '다른' '다만' '다소' '다시' '다시금' '다운' '다음' '다큐' '다행' '닥터' '단']


In [28]:
# 새로운 텍스트의 tfidf 특징 벡터 추출
print(review_tv.transform(["이 영화 굿~~~~"]))

  (np.int32(0), np.int32(95))	0.7876359164275684
  (np.int32(0), np.int32(595))	0.29060640671112975
  (np.int32(0), np.int32(666))	0.5433024751754311


**[참고] Tfidf 특징 벡터 추출 모델을 파일로 저장하기**

In [29]:
import joblib
joblib.dump(review_tv, './model/daum_review_tv.pkl')

['./model/daum_review_tv.pkl']

In [30]:
loaded_tv = joblib.load('./model/daum_review_tv.pkl')

In [31]:
print(loaded_tv.transform(["이 영화 굿~~~~"]))

  (np.int32(0), np.int32(95))	0.7876359164275684
  (np.int32(0), np.int32(595))	0.29060640671112975
  (np.int32(0), np.int32(666))	0.5433024751754311
